# ROPE Demo on Google Colab

Run the ROPE demo on a Colab GPU and download the results.

**Before starting:**
1. **Runtime -> Change runtime type -> GPU** (T4 or A100).
2. Get a [HuggingFace token](https://huggingface.co/settings/tokens) if you want to use Llama (phi2 works without it).

**Option A:** You have the ROPE repo on GitHub -> set `REPO_URL` below and run the clone cell.

**Option B:** No GitHub -> zip your local ROPE folder, upload it in cell 2, then run the rest.

## 1. Install dependencies and clone or use uploaded ROPE

In [ ]:
import os

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU. Set Runtime -> Change runtime type -> GPU.")

# Option A: Clone from GitHub (set your repo URL)
REPO_URL = "https://github.com/yourusername/ROPE.git"  # <-- Change to your repo
REPO_DIR = "ROPE"

if not os.path.isdir(REPO_DIR):
    if "yourusername" in REPO_URL:
        print("Set REPO_URL to your repo, or upload a zip of the ROPE folder (Option B below).")
    else:
        !git clone --depth 1 "{REPO_URL}" "{REPO_DIR}"

if os.path.isdir(REPO_DIR):
    %cd {REPO_DIR}
    !pip install -e . -q
    print("ROPE installed.")
else:
    print("ROPE folder not found. Upload a zip in the next cell (Option B).")

## 2. Option B: Upload ROPE zip (if you didn't clone)

Zip your local `ROPE` project folder (include `rope/`, `data/`, `pyproject.toml`, etc.), upload below, then re-run cell 1 logic in the next cell.

In [ ]:
# Run this only if you're uploading a zip. Upload the zip when prompted.
from google.colab import files
import zipfile

uploaded = files.upload()  # Opens file picker
for name in uploaded:
    if name.endswith('.zip'):
        with zipfile.ZipFile(name, 'r') as z:
            z.extractall('.')
        print(f"Extracted {name}")
        break
# Go to the folder that contains pyproject.toml
import os
for d in ['ROPE', '.', 'rope-bench']:
    if os.path.isfile(os.path.join(d, 'pyproject.toml')):
        %cd {d}
        !pip install -e . -q
        print(f"Installed from {d}")
        break

## 3. HuggingFace login (for Llama models)

In [ ]:
from getpass import getpass
from huggingface_hub import login

token = getpass("Paste your HuggingFace token (or press Enter to skip; phi2 works without it): ")
if token.strip():
    login(token=token.strip())
    print("Login successful. Gated models (Llama) will now work.")
else:
    print("Skipped. Use phi2-only runs (e.g. rope run --models phi2) or add token for Llama.")

## 4. Run ROPE demo

Add `--verbose` to save full debug info (raw judge outputs, defended prompts, etc.).

In [ ]:
# Standard demo: llama2-7b + 2 defenses + 20 attacks (~5-15 min on T4)
# Add --verbose for full debug output
!rope demo --verbose

## 5. View results

In [ ]:
import json
import pandas as pd
import os

with open("demo_results.json") as f:
    results = json.load(f)

# Load or compute metrics
if os.path.exists("demo_results_metrics.csv"):
    metrics = pd.read_csv("demo_results_metrics.csv")
else:
    from rope.metrics import compute_metrics
    metrics = compute_metrics(results)
    print("Note: CSV not found, computed metrics from JSON.")

print("=" * 70)
print("OVERALL METRICS")
print("=" * 70)
display(metrics)

# Per-attack-type breakdown
from rope.metrics import compute_by_attack_type, compute_by_task_family
by_type = compute_by_attack_type(results)
print("\n" + "=" * 70)
print("BREAKDOWN BY ATTACK TYPE")
print("=" * 70)
display(by_type)

# Per-task-family breakdown
by_family = compute_by_task_family(results)
print("\n" + "=" * 70)
print("BREAKDOWN BY TASK FAMILY")
print("=" * 70)
display(by_family)

print(f"\nTotal evaluations: {len(results)}")
print("\nSample result:")
print(json.dumps(results[0], indent=2))

## 6. Inspect judge outputs (debug)

Check what the judge model actually said to verify scoring.

In [ ]:
# Show judge outputs for the first 10 results
for i, r in enumerate(results[:10]):
    judge = r.get('judge_output', 'N/A')
    print(f"[{i}] severity={r['severity']}  judge='{judge}'  type={r['attack_type']}  model={r['model']}/{r['defense']}")

# Count severity distribution
from collections import Counter
sev_dist = Counter(r['severity'] for r in results)
print(f"\nSeverity distribution: {dict(sorted(sev_dist.items()))}")

## 7. Download result files

In [ ]:
from google.colab import files

for name in ["demo_results.json", "demo_results_metrics.csv", "demo_results_report.txt"]:
    if os.path.exists(name):
        files.download(name)
        print(f"Downloaded {name}")
    else:
        print(f"Not found: {name}")